In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50
from tensorflow.keras.layers import (Input, GlobalAveragePooling2D, Dense, Dropout, concatenate)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping


csv_path = "C:/Users/ajays/Downloads/petfinder-pawpularity-score/train.csv"
df = pd.read_csv(csv_path)

if not df["Id"].iloc[0].endswith(".jpg"):
    df["Id"] = df["Id"].astype(str) + ".jpg"

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)


class PawpularitySequence(Sequence):
    
    def __init__(self,
                 df,
                 image_dir,
                 batch_size=32,
                 target_size=(224, 224),
                 shuffle=True,
                 augment=False):
       
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        self.augment = augment
        
        self.indexes = np.arange(len(self.df))
        
        self.filenames = self.df['Id'].values
        self.labels = self.df['Pawpularity'].values.astype('float32')
        
        self.numeric_cols = [
            "Subject Focus","Eyes","Face","Near","Action","Accessory",
            "Group","Collage","Human","Occlusion","Info","Blur"
        ]
        self.numeric_data = self.df[self.numeric_cols].values.astype('float32')
        
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size : (index+1)*self.batch_size]
        
        batch_filenames = self.filenames[batch_indexes]
        batch_labels = self.labels[batch_indexes]
        batch_numeric = self.numeric_data[batch_indexes]
        
        images = [self._load_image(fn) for fn in batch_filenames]
        
        images = np.array(images, dtype='float32')
        
        return [images, batch_numeric], batch_labels

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def _load_image(self, filename):
        img_path = os.path.join(self.image_dir, filename)
        img = load_img(img_path, target_size=self.target_size)
        img = img_to_array(img)
        
        img = preprocess_input(img)
        
        if self.augment:
            img = self._augment_image(img)
        return img
    
    def _augment_image(self, img):
        img_tensor = tf.convert_to_tensor(img, dtype=tf.float32)
        
        img_tensor = tf.image.random_flip_left_right(img_tensor)
        img_tensor = tf.image.random_flip_up_down(img_tensor)
        img_tensor = tf.image.random_brightness(img_tensor, max_delta=0.1)
        img_tensor = tf.image.random_contrast(img_tensor, lower=0.8, upper=1.2)
        
        return img_tensor.numpy()


image_input = Input(shape=(224, 224, 3), name="image_input")
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=image_input)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)

numeric_input = Input(shape=(12,), name="numeric_input")
y = Dense(64, activation='relu')(numeric_input)
y = Dropout(0.5)(y)

combined = concatenate([x, y])
combined = Dense(128, activation='relu')(combined)
combined = Dropout(0.5)(combined)

output = Dense(1, activation='linear')(combined)

model = Model(inputs=[image_input, numeric_input], outputs=output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='mean_squared_error',
    metrics=['mae']
)

model.summary()


image_folder = "C:/Users/ajays/Downloads/petfinder-pawpularity-score/train"

train_seq = PawpularitySequence(
    df=train_df,
    image_dir=image_folder,
    batch_size=32,
    target_size=(224, 224),
    shuffle=True,
    augment=True   
)

val_seq = PawpularitySequence(
    df=val_df,
    image_dir=image_folder,
    batch_size=32,
    target_size=(224, 224),
    shuffle=False,
    augment=False  
)


checkpoint = ModelCheckpoint(
    "best_model.h5", 
    monitor="val_loss", 
    save_best_only=True, 
    verbose=1
)
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", 
    factor=0.5, 
    patience=3, 
    verbose=1
)
early_stopping = EarlyStopping(
    monitor="val_loss", 
    patience=5, 
    restore_best_weights=True, 
    verbose=1
)


history = model.fit(
    train_seq,
    validation_data=val_seq,
    epochs=10,  # short first stage
    callbacks=[checkpoint, reduce_lr, early_stopping],
    verbose=1
)


unfreeze_layers = 20
for layer in base_model.layers[-unfreeze_layers:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='mean_squared_error',
    metrics=['mae']
)

model.summary()  

history_finetune = model.fit(
    train_seq,
    validation_data=val_seq,
    epochs=10,  
    callbacks=[checkpoint, reduce_lr, early_stopping],
    verbose=1
)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_input (InputLayer)       [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['image_input[0][0]']            
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input



class PawpularitySequence(tf.keras.utils.Sequence):
    def __init__(self,
                 df,
                 image_dir,
                 batch_size=32,
                 target_size=(224, 224),
                 shuffle=True,
                 augment=False):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        self.augment = augment
        
        self.indexes = np.arange(len(self.df))
        
        self.filenames = self.df['Id'].values
        self.labels = self.df['Pawpularity'].values.astype('float32')
        
        self.numeric_cols = [
            "Subject Focus","Eyes","Face","Near","Action","Accessory",
            "Group","Collage","Human","Occlusion","Info","Blur"
        ]
        self.numeric_data = self.df[self.numeric_cols].values.astype('float32')
        
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size : (index+1)*self.batch_size]
        
        batch_filenames = self.filenames[batch_indexes]
        batch_labels = self.labels[batch_indexes]
        batch_numeric = self.numeric_data[batch_indexes]
        
        images = [self._load_image(fn) for fn in batch_filenames]
        images = np.array(images, dtype='float32')
        
        return [images, batch_numeric], batch_labels

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def _load_image(self, filename):
        img_path = os.path.join(self.image_dir, filename)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=self.target_size)
        img = tf.keras.preprocessing.image.img_to_array(img)
        
        img = preprocess_input(img)
        
        if self.augment:
            img = self._augment_image(img)
        return img
    
    def _augment_image(self, img):
        img_tensor = tf.convert_to_tensor(img, dtype=tf.float32)
        
        img_tensor = tf.image.random_flip_left_right(img_tensor)
        img_tensor = tf.image.random_flip_up_down(img_tensor)
        img_tensor = tf.image.random_brightness(img_tensor, max_delta=0.1)
        img_tensor = tf.image.random_contrast(img_tensor, lower=0.8, upper=1.2)
        
        return img_tensor.numpy()


def build_mobilenet_model(img_shape=(224, 224, 3), num_numeric_features=12):
    
    image_input = Input(shape=img_shape, name="image_input")
    base_model = MobileNetV2(weights='imagenet', 
                             include_top=False, 
                             input_tensor=image_input)

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    numeric_input = Input(shape=(num_numeric_features,), name="numeric_input")
    y = Dense(64, activation='relu')(numeric_input)
    y = Dropout(0.5)(y)

    combined = concatenate([x, y])
    combined = Dense(128, activation='relu')(combined)
    combined = Dropout(0.5)(combined)

    output = Dense(1, activation='linear')(combined)

    model = Model(inputs=[image_input, numeric_input], outputs=output)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='mean_squared_error',
        metrics=['mae']
    )
    return model


def train_mobilenet_model(train_df, val_df, image_dir, 
                          batch_size=32, target_size=(224, 224), epochs=10):
    model = build_mobilenet_model(img_shape=(target_size[0], target_size[1], 3),
                                  num_numeric_features=12)
    model.summary()

    train_seq = PawpularitySequence(
        df=train_df,
        image_dir=image_dir,
        batch_size=batch_size,
        target_size=target_size,
        shuffle=True,
        augment=True  
    )

    val_seq = PawpularitySequence(
        df=val_df,
        image_dir=image_dir,
        batch_size=batch_size,
        target_size=target_size,
        shuffle=False,
        augment=False  
    )

    checkpoint = ModelCheckpoint(
        "mobilenet_model_best.h5",
        monitor="val_loss",
        save_best_only=True,
        verbose=1
    )
    reduce_lr = ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        verbose=1
    )
    early_stop = EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True,
        verbose=1
    )

    history = model.fit(
        train_seq,
        validation_data=val_seq,
        epochs=epochs,
        callbacks=[checkpoint, reduce_lr, early_stop],
        verbose=1
    )


    model.save("mobilenet_model_final.h5")
    
    return model, history


In [4]:
model, history = train_mobilenet_model(
    train_df=train_df,
    val_df=val_df,
    image_dir="C:/Users/ajays/Downloads/petfinder-pawpularity-score/train",
    batch_size=32,
    target_size=(224, 224),
    epochs=10
)


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_input (InputLayer)       [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['image_input[0][0]']            
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [ ]:
def load_trained_models(resnet_path="best_model.h5",
                        lightweight_path="mobilenet_model_best.h5"):
    
    resnet_model = tf.keras.models.load_model(resnet_path, compile=False)
    light_model = tf.keras.models.load_model(lightweight_path, compile=False)
    
    resnet_model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-5),
        loss='mean_squared_error',
        metrics=['mae']
    )
    light_model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-5),
        loss='mean_squared_error',
        metrics=['mae']
    )
    
    return resnet_model, light_model

def ensemble_predict(resnet_model, light_model, data_sequence, alpha=0.5):
    
    preds_resnet = resnet_model.predict(data_sequence)
    preds_light  = light_model.predict(data_sequence)
    
    ensemble_preds = alpha * preds_resnet + (1 - alpha) * preds_light
    return ensemble_preds

def run_ensemble_example(val_df, image_dir):
    
    val_seq = PawpularitySequence(
        df=val_df,
        image_dir=image_dir,
        batch_size=32,
        target_size=(224, 224),
        shuffle=False,
        augment=False
    )
    
    resnet_model, light_model = load_trained_models(
        resnet_path="best_model.h5", 
        lightweight_path="mobilenet_model_best.h5"
    )
    
    y_ensemble = ensemble_predict(resnet_model, light_model, val_seq, alpha=0.5)
    
    y_true = val_df['Pawpularity'].values
    mae_ensemble = np.mean(np.abs(y_true - y_ensemble.squeeze()))
    mse_ensemble = np.mean((y_true - y_ensemble.squeeze())**2)
    
    print(f"Ensemble MAE: {mae_ensemble:.4f}")
    print(f"Ensemble MSE: {mse_ensemble:.4f}")

    return y_ensemble


In [8]:
y_ens = run_ensemble_example(val_df, image_dir="C:/Users/ajays/Downloads/petfinder-pawpularity-score/train")


62/62 [==============================] - 10s 147ms/step
Ensemble MAE: 14.9984
Ensemble MSE: 479.7789
